# [KB] 네이버 카페 크롤러 with 키워드 입력_게시판지정_날짜지정

### 환경설정

In [ ]:
!pip install selenium==4.1.0 webdriver-manager

### 검색 조건 입력

In [13]:
# 키워드 설정 - 단독 키워드 or 결합 키워드(네이버 검색 연산자 사용) 입력

keywords = ['절세','환급','지출증빙','비과세','공제','보양가족','매입세액','지방세','부가세']

# 날짜 지정
start_date = '2019-01-01'
end_date = '2023-08-31' 

In [14]:
data_path = f'./data/[KB]셀러오션_{keywords[0]}-{keywords[-1]}.csv'

### 라이브러리 import

In [15]:
import pandas as pd
import numpy as np
import random

from bs4 import BeautifulSoup as bs
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from webdriver_manager.chrome import ChromeDriverManager
import time
from time import sleep

### 네이버 로그인

In [16]:
# 크롬 드라이버 제어
driver = webdriver.Chrome(ChromeDriverManager().install()) #현재 컴퓨터 크롬드라이버 위치로 변경 114.0.5735.199


# 네이버 로그인 화면 이동
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)


# 아이디& 비밀번호 입력
my_id = 'tinacham' # "id" 대신에 자신의 네이버 아이디 입력
my_pw = 'Ckdgus1228!!' # "password" 대신에 자신의 네이버 비밀번호 입력


# 로그인 id, pw 입력
# 네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 함수를 사용
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)


# '로그인' 버튼 클릭
driver.find_element('id', 'log.login').click()
time.sleep(1)


/var/folders/rm/kw9zwc010ng_sj6xl0nq6vlw0000gn/T/ipykernel_14357/2917560162.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #현재 컴퓨터 크롬드라이버 위치로 변경 114.0.5735.199



### 크롤링할 카페 접속 및 원하는 게시판 클릭 & 게시물 크롤링

In [ ]:
# 제목, 본문, 댓글, 날짜, 검색 키워드, 게시판 이름 - 빈 list 생성
titles = []
reviews = []
comments = []
dates = []
search_keywords = []
board_titles = []

board = ['//*[@id="menuLink90"]'] # 크롤링할 게시판의 xpath 입력; 원하는 게시판 클릭 -> F12 눌러 게시판 선택자 확인 가능

# 네이버 카페 접속
# 로그인한 네이버 계정이 해당 카페-게시판에 접근 권한이 있어야 합니다.(회원가입 & 등업)
# 프레임 전환(네이버 카페는 여러 프레임으로 구성되어서 크롤링할 게시글이 있는 프레임으로 전환해주어야 함)

time.sleep(random.uniform(1,1.7))
url = 'https://cafe.naver.com/soho' # 크롤링할 카페 url 입력
driver.get(url)
time.sleep(1)


time.sleep(1)
for k in keywords:
    for i in board:
        #원하는 게시판 클릭
        driver.find_element_by_xpath('%s'%i).click()
        time.sleep(1)
        driver.switch_to.frame('cafe_main') # 프레임 전환

        #검색
        search_box = driver.find_element(By.XPATH, '//*[@id="query"]')
        search_box.send_keys(k)
        search_box.send_keys(Keys.RETURN)
        time.sleep(3)
        # 3초간 대기 후 다음 코드 실행
        #검색결과 없을 때 다음 게시판으로 넘어가도록
        try:
            if driver.find_element(By.XPATH, '//*[@id="main-area"]/div[5]/table/tbody/tr/td/div').text.strip() == '등록된 게시글이 없습니다.':
                driver.get(url)
                time.sleep(1)
                continue

        except:
            pass
        time_box = driver.find_element(By.XPATH, '//*[@id="currentSearchDateTop"]')
        time_box.click()
        start_time = driver.find_element(By.XPATH, '//*[@id="input_1_top"]')
        start_time.click()
        time.sleep(0.5)
        start_time.send_keys(start_date)
        time.sleep(1)
        end_time = driver.find_element(By.XPATH, '//*[@id="input_2_top"]')
        end_time.click()
        time.sleep(0.5)
        end_time.send_keys(end_date)
        driver.find_element(By.XPATH, '//*[@id="btn_set_top"]').click()
        time.sleep(1)




        driver.find_element(By.XPATH, '//*[@id="main-area"]/div[1]/div[1]/form/div[4]/button').click()



        try:
            if driver.find_element(By.XPATH, '//*[@id="main-area"]/div[5]/table/tbody/tr/td/div').text.strip() == '등록된 게시글이 없습니다.':
                driver.get(url)
                time.sleep(1)
                continue

        except:
            print("크롤링 시작합미당~")
            pass

        # 50개씩 보기
        driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click()
        driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()

        #url 가져오기
        page_url_list = [a.get_attribute('href') for a in driver.find_elements_by_css_selector('a.article')]
        first_page = driver.find_element_by_css_selector('a.on').get_attribute('href')
        #페이지 넘기기 검색시 최대 100페이지
        for j in range(1,101):
            #400개마다 껐다 키기
            try:
                if j % 8 == 0:
                    #다음 링크 가져오기
                    time.sleep(1)
                    link = first_page[:-1] + str(j)


                    driver.quit()
                    driver = webdriver.Chrome(ChromeDriverManager().install()) #현재 컴퓨터 크롬드라이버 위치로 변경


                    ##재로그인
                    # 네이버 로그인 화면 이동
                    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
                    driver.get(login_url)
                    driver.implicitly_wait(10)


                    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
                    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
                    time.sleep(1)


                    # '로그인' 버튼 클릭
                    driver.find_element('id', 'log.login').click()
                    time.sleep(1)

                    #다음 링크부터 가져오기
                    driver.get(link)
                    driver.switch_to.frame('cafe_main')

            except:
                pass
                #결과 없을 때 넘어가기
            try:
                if driver.find_element(By.XPATH, '//*[@id="main-area"]/div[5]/table/tbody/tr/td/div').text.strip() == '등록된 게시글이 없습니다.':
                    break
            except:
                pass
            if j == 1:
                pass
            else:
                next_page = first_page[:-1] + str(j)
                driver.get(next_page)
                time.sleep(1)
                driver.switch_to.frame('cafe_main')

            page_url_list = [a.get_attribute('href') for a in driver.find_elements_by_css_selector('a.article')]


            for url in page_url_list:
                driver.get(url)
                time.sleep(random.uniform(1,1.7))
                driver.switch_to.frame('cafe_main')
                try:
                    #키워드 수집
                    search_keywords.append(k)

                    #게시판 이름 수집
                    board_title = driver.find_element(By.CSS_SELECTOR, 'div.ArticleTitle').text
                    board_titles.append(board_title)
                    title = driver.find_element(By.CSS_SELECTOR, 'h3.title_text').text
                    titles.append(title)

                    #날짜 수집
                    day = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div.profile_area > div.article_info > span.date').text
                    soup = bs(driver.page_source, 'lxml')
                    isdate = soup.find_all('span',class_='date')
                    dates.append(isdate)

                except:
                    continue
                try:
                    content = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text').text
                    reviews.append(content)
                except:
                    try:
                        content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer').text
                        reviews.append(content)
                    except:
                        try:
                            content = driver.find_element(By.CSS_SELECTOR, 'div.scrap_added').text.strip()
                            reviews.append(content)
                        except:
                            content= [] #게시물 본문이 사진으로만 구성되어있을때
                            reviews.append(content)

                # 댓글 수집
                soup = bs(driver.page_source, 'lxml')
                iscomment = soup.find_all('span',class_='text_comment')
                if len(iscomment) == 0:
                    box = []

                else:
                    WebDriverWait(driver, 15).until( EC.presence_of_all_elements_located((By.CLASS_NAME, "text_comment")) )

                    soup = bs(driver.page_source, 'lxml')
                    iscomment = soup.find_all('span',class_='text_comment')
                    box = []
                    for i in iscomment:
                        box.append([i.get_text()])

                comments.append(box)


        url = 'https://cafe.naver.com/soho' # 크롤링할 카페 url 입력
        driver.get(url)
        time.sleep(1)

# 크롬 창 닫기
driver.quit()


### 크롤링 개수 확인(키워드, 게시판, 제목, 본문, 댓글, 날짜)

In [ ]:
print(len(search_keywords))
print(len(board_titles))
print(len(titles))
print(len(reviews))
print(len(comments))
print(len(dates))

### 크롤링 데이터 합치기

In [ ]:
# 날짜 포멧 '2023.03.20.' 위 형태로 포맷
import re

formatted_dates = []
#게시판 제목 포맷
formatted_boards = []
for board_title in board_titles:
    formatted_boards.append(board_title.split("\n")[0])


for date in [str(d[0]) for d in dates]:
    date_str = re.search(r'\d{4}\.\d{2}\.\d{2}\.', date).group(0)
    formatted_dates.append(date_str)

# ===데이터 합치기===

data = {'keyword' : search_keywords,
        'board' : formatted_boards,
        'tit' : titles,
        'body' : reviews,
        'comment' : comments,
        'date': formatted_dates}

dataDF = pd.DataFrame(data)

### CSV 저장 & 데이터 확인

In [ ]:
dataDF

#csv로 저장
dataDF.to_csv(data_path, encoding = "utf-8-sig", index=False)

In [ ]:
dataDF

,keyword,board,tit,body,comment,date
0,24시간,후기(신혼집),[Wedding Ch.3] 집 계약 (깐깐한 신부의 반전 총알계약),"★\n※다이렉트결혼준비 비제휴업체의 홍보글, 후기글은 자동 삭제처리 될 수 있으며 ...",[[집에 제일 어려운 문제인것 같아요 ㅎㅎ 저도 아직 알아보지는 않았지만엄청 꼼꼼하...,2022.06.13.
1,24시간,후기(신혼집),♥ 신혼집 랜선집들이 ♥,다들 신혼집에 대한 로망 있으시잖아요? ㅎㅎ\n그래서 저도 열심히 꾸몄었는데 .. ...,"[[신혼집 깔끔하게 잘 꾸며놓으신 것 같아용ㅎㅎ], [집이 깔끔하고 색도 맞추신것같...",2022.03.21.
2,24시간,후기(신혼집),매트리스 구매,매트리스가 필요해서 인터넷으로 검색하다 남이면에 괜찮은 매장이 있다고 확인해서 가봤...,"[[저희도 이태리 제품 매트리스햇눈대 좋더라고요 ㅎㅎ], [지역만 가까우면 저도 누...",2022.01.07.
3,24시간,후기(신혼집),보관이사했어용~,드디어 이사 끝나구 근 한달 다되어가네요 ㅎㅎ\n갑자기 이사하는바람에 사실 여러군데...,"[[오 보관이사도 있군요 처음 알았어요. 나중에 유용하게 쓸 듯 합니다], [보관이...",2021.09.06.
4,24시간,후기(신혼집),신혼집 입주 후기입니다!!,신혼집 자금마련 후기와 신혼집 결정 후기와 이어서 입주후기 남기려고 합니당~^^ 저...,"[[와 벽에 자석 붙인 수납장? 저건 뭔가용 너무 탐나요!], [입주축하드려용 행복...",2020.12.31.
5,24시간,후기(신혼집),신혼집 결정 및 결정후기입니다 ღ,저희 신혼집 결정후기입니다:)\n남편이 결혼준비할때 집에 대한 부분을 가장 많이 신...,"[[축하드립니다 신혼집에서행복하세요], [오오 저희도 노원 알아보는데 여긴 어딘가요...",2020.12.31.
6,24시간,후기(신혼집),신혼집 입주 후기입니다!,안녕하세요! 내년 5월 예식을 앞둔 예신입니다.\n신혼집 계약 후 입주하여 입주 후...,[[뷰가 예술이네요 정말! 테이블에 앉아서 식사하시면 고급레스토랑 온 느낌일거 같아...,2020.12.31.
7,24시간,후기(신혼집),신혼집 입주 후기 입니다,신혼집 입주 후기 입니다.\n시흥 대야동에 위치안 새아파트에 입주하게된 저희 커플은...,"[[큰 일 치루셨네요! 입주 축하드립니다!], [어머나 야경이 너무 멋져용 축하드려...",2020.12.30.
8,24시간,후기(신혼집),신혼집 결정이유 (feat.예랑이노력),안녕하세요 가을의 신부 준비로 한창인 요즘이네요^^\n신혼집은 식 한달전에 인테리어...,"[[신축이라 좋으시겠어요! 축하드립니다], [오 집 넘 좋네요ㅎ그것도 신축~~!축하...",2020.12.20.
9,24시간,후기(신혼집),신혼집선정이유후기입니다~,안녕하세요~ 조두순출소로 비상이네요ㅜ 마음편해지는 세상이 왔음좋겠어요ㅜ\n저희 신혼...,"[[너무너무수고하셨어요!!^^ 이사까지 잘마무리되시길^^], [스세권 좋네요^^ ...",2020.12.16.
